<a href="https://colab.research.google.com/github/Praxis-bs/DEMD/blob/main/WebApps_for_ML_in_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://4.bp.blogspot.com/-gbL5nZDkpFQ/XScFYwoTEII/AAAAAAAAAGY/CcVb_HDLwvs2Brv5T4vSsUcz7O4r2Q79ACK4BGAYYCw/s1600/kk3-header00-beta.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

#Introduction

This exercise is loosely modelled on the tutorial given in https://pythonspot.com/flask-web-app-with-python/ except that instead of working with a local machine where the TCPIP ports are easily accessible, we use Google Colab where such ports are not easily accessible. To overcome this problem we use ngrok tunnelling technology that creates a publicly accessible URL that can access TCPIP applications running on any Google Colab VM port <br>
All files used in these exercises are availale at https://drive.google.com/drive/folders/1SzeS5AqnCNa9FTSeUHCGcJsClrGNlGVt?usp=sharing <br> <br>
Other ways of achieving this are as follows 


*   flask-ngrok - https://pypi.org/project/flask-ngrok/
*   pyngrok - https://pypi.org/project/pyngrok/
<br>
However the MOST SIMPLEST way to access a port in google colab is to follow the instructions given in <br>
 https://stackoverflow.com/questions/59741453/is-there-a-general-way-to-run-web-applications-on-google-colab <br>

see https://colab.research.google.com/drive/1XIz5OE3CqvlqjyYPQxrO-zhlVa8ZhMA0?usp=sharing


#ngrok configuration

ngrok needs an authentication token. this can be easily hardcoded into this notebook ( as shown in one of the cells below) To avoid hardcoding -- and revealing the same in a public notebook -- the token is stored in Google Drive and accessed by mounting the G-Drive on the Colab VM <br>
See 


1.   https://ngrok.com/ - login 
2.   https://dashboard.ngrok.com/get-started/setup - get your authtoken



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#!ls /content/drive/'My Drive'/Praxis/WebCredentials
!cp /content/drive/'My Drive'/Praxis/WebCredentials/ngrokToken.py .

In [3]:
# credential file for Prithwis Mukerjee
# this file needs to be uploaded into the VM

from ngrokToken import ngrokToken

#for the sake of privacy
#the following credentials need to be stored in a text file called ngrokToken.py
#in the format given below
#in the Colab VM

#otherwise, the values can be directly placed here

#ngrokToken = '1kUOcwwwwwwwwRHsdVJRGMeTdddddddddddYbPZhxod'   # uncomment this line and place your own credentials here
#token is available at https://dashboard.ngrok.com/get-started/setup


#print(ngrokToken)
ngrokTokenCmd = 'ngrok authtoken '+ngrokToken

In [4]:
#https://ngrok.com/download
!wget -q https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip     # -q quiet mode, remove to see errors if any

In [5]:
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [6]:
#!./ngrok authtoken xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
get_ipython().system_raw(ngrokTokenCmd)

In [7]:
!apt-get install jq > /dev/null

#WebApp Linear Regression - Wine Analysis

Web apps need additionl files for HTML, images, data etc that need to be stored on appropriate directories. These files can either be created in a local machine and uploaded into the appropriate directory, or loaded from GDrive or Github

In [8]:
!mkdir static
!mkdir templates
!mkdir datafiles

In [9]:
!wget -q https://github.com/Praxis-bs/DEMD/raw/main/static/kkpraxis.png      # image file

!wget -q https://raw.githubusercontent.com/Praxis-bs/DEMD/main/templates/LR_home.html    # home
!wget -q https://raw.githubusercontent.com/Praxis-bs/DEMD/main/templates/LR_predict1.html  # output

!wget -q https://github.com/Praxis-bs/DEMD/raw/main/datafiles/lrw-model.pkl # model file

In [10]:
!mv kkpraxis.png static
!mv *.html templates/
!mv *.pkl datafiles/

In [11]:
get_ipython().system_raw('./ngrok http 5010 &')

In [12]:
!curl -s http://localhost:4040/api/tunnels | jq .tunnels[0].public_url
!curl -s http://localhost:4040/api/tunnels | jq .tunnels[0].config.addr

"https://aa08-35-231-236-132.ngrok.io"
"http://localhost:5010"


In [13]:
# https://pythonspot.com/flask-web-app-with-python/
# COLAB where model is trained, tested
# Train : https://colab.research.google.com/drive/1G3VUKVUspcoDnLRobuUfsGTswpCMkU8W
# Predict : https://colab.research.google.com/drive/1gF9DI4NT5tH_9lcbVth-kVEZ9sO7wnhU
# docker build -t calcutta/de-applrw02 .
# docker run --rm --name praxislrw -p 5002:5000 calcutta/de-applrw02

from sys import argv
from flask import Flask, flash, redirect, render_template, request, session, abort

import pickle
from sklearn import linear_model

app = Flask(__name__)
scriptName = argv[0]

@app.route("/")
def hello():
    retString = 'ML LRW now running ... '+scriptName
    return retString

@app.route('/home')
def home():
    return render_template('LR_home.html')



@app.route('/predict1/<featureString>')
def multidata(featureString):
    str1 = featureString
    feature_list = featureString.split(",")
    feature_array = list(map(lambda x : float(x), feature_list))

    model = pickle.load(open("datafiles/lrw-model.pkl","rb"))

    #our model rates the wine based on the input array
    prediction = model.predict([feature_array]).tolist()

    return render_template('LR_predict1.html',**locals())


if __name__ == "__main__":
    app.run(host="localhost", port=5010)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:5010/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Aug/2021 23:22:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/2021 23:22:36] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [24/Aug/2021 23:22:43] "GET /home HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/2021 23:22:43] "GET /static/kkpraxis.png HTTP/1.1" 200 -


#WebApp for ML - Image Classification <br>
https://blog.hyperiondev.com/index.php/2018/02/01/deploy-machine-learning-model-flask-api/

In [14]:
# the model is retrieved from G-Drive
#!cp /content/drive/'My Drive'/Praxis/'Course - DevOps'/DevOps/ml/appimclass02/datafiles/*.pkl datafiles
#https://drive.google.com/file/d/1o6Z02Q-KATgEXSGbQ8XK_H62Dv7BW4eW/view?usp=sharing
!gdown https://drive.google.com/uc?id=1o6Z02Q-KATgEXSGbQ8XK_H62Dv7BW4eW

#or it can be rebuilt again from this notebook https://github.com/Praxis-bs/DEMD/blob/main/Image_Classification_Train_%26_Test.ipynb

# the html file 

!wget -q https://raw.githubusercontent.com/Praxis-bs/DEMD/main/templates/IC_index.html

Downloading...
From: https://drive.google.com/uc?id=1o6Z02Q-KATgEXSGbQ8XK_H62Dv7BW4eW
To: /content/imClass-model-18032020.pkl
336MB [00:02, 151MB/s]


In [15]:
!mv *.pkl datafiles/
!mv *index.html templates/

In [16]:
get_ipython().system_raw('./ngrok http 5010 &')

In [17]:
!curl -s http://localhost:4040/api/tunnels | jq .tunnels[0].public_url
!curl -s http://localhost:4040/api/tunnels | jq .tunnels[0].config.addr

"https://0767-35-231-236-132.ngrok.io"
"http://localhost:5010"


In [18]:
# https://pythonspot.com/flask-web-app-with-python/
# https://blog.hyperiondev.com/index.php/2018/02/01/deploy-machine-learning-model-flask-api/
# COLAB for train, test, model build https://colab.research.google.com/drive/1JuTezUiqM0Cj_m-FmmeBzw_88xZlYeK-
# docker build -t calcutta/de-appimc02 .
# docker run --rm --name praxisimc -p 5000:5000 calcutta/de-appimc02

# test images available at https://github.com/elliebirbeck/model-deployment-flask/tree/master/test-imgs


from sys import argv
from flask import Flask, flash, redirect, render_template, request, session, abort, url_for

import pickle
from sklearn import linear_model
#from sklearn.externals import joblib
import joblib
from sklearn.ensemble import *

import numpy as np
import imageio


app = Flask(__name__)
scriptName = argv[0]

@app.route("/")
def hello():
    retString = 'now running ... '+scriptName
    return retString

@app.route('/home')
def home():
    return render_template('IC_index.html')

@app.route('/predict', methods=['POST'])
def make_prediction():
    if request.method=='POST':
        file = request.files['image']
        if not file: 
            return render_template('IC_index.html', label="No file")
        #f = 'datafiles/'+str(file.filename)
        #file.save(f)
        
        img = imageio.imread(file)
        img = img[:,:,:3]
        img = img.reshape(1, -1)
        prediction = model.predict(img)
        label = str(np.squeeze(prediction))
        if label=='10': 
            label='0'
        return render_template('IC_index.html', label=label, file=file)

if __name__ == "__main__":
    model = joblib.load('datafiles/imClass-model-18032020.pkl')
    app.run(host="localhost", port=5010)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:5010/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Aug/2021 23:45:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/2021 23:45:02] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [24/Aug/2021 23:45:12] "GET /home HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/2021 23:45:12] "GET /static/kkpraxis.png HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/2021 23:46:03] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/2021 23:46:10] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/2021 23:46:24] "POST /predict HTTP/1.1" 200 -


#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself. 
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)